<a href="https://colab.research.google.com/github/abhirbhandary/Data_Engineering/blob/main/Data_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aim

1.   To Extract Data by Scraping Wikipedia
2.   To Get get using API
3.   To Save the data in a database



### 1. Extracting Data from Wikipedia about Franfurt.

In [ ]:
# colab has an older version of beautifulsoup by default
# here we upgrade it
# if you are working on your own computer, you can probably comment this step out and skip it
!pip install --upgrade beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2


In [ ]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/Frankfurt_am_Main"

In [ ]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [ ]:
#headers = {'Accept-Language': 'en-US,en;q=0.8'}
#response = requests.get(url, headers = headers)

In [ ]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-disabled vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Frankfurt - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-cl

In [ ]:
# soup

In [ ]:
# 5. retrieve/extract the desired info
# let's first try to get the name of the city
# by copying the selector we can see that it has the id firstHeading (it also has a class by the same name!)
soup.select("#firstHeading")

[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Frankfurt</span></h1>]

In [ ]:
soup.select("#firstHeading")[0]

<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Frankfurt</span></h1>

In [ ]:
soup.select("#firstHeading")[0].get_text()

'Frankfurt'

In [ ]:
# here we initialise our empty dictionary for the city
response_dict = {}

# here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
response_dict['city'] = soup.select(".firstHeading")[0].get_text()
response_dict['country'] = soup.select(".infobox-data")[0].get_text()
response_dict['latitude'] = soup.select(".latitude")[0].get_text()
response_dict['longitude'] = soup.select(".longitude")[0].get_text()
response_dict['Population'] = soup.select(".infobox-data")[10].get_text()

In [ ]:
# add our dictionary for the city to list_for_df
list_for_df = []
list_for_df.append(response_dict)

# make the DataFrame
cities_df = pd.DataFrame(list_for_df)

In [ ]:
  # fixing latitude
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing longitude
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)

In [ ]:
# fixing Population
cities_df['Population'] = cities_df['Population'].str.replace(',', '', regex=False)

In [ ]:
cities_df

,city,country,latitude,longitude,Population
0,Frankfurt,Germany,50.0638,08.4056,773068


### 2. Open Weather API for Franfurt.

In [ ]:
import json

In [ ]:
# Custom Function to get data like IPython.display in jupiter notebook
from IPython.display import HTML

def render_json(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)

In [ ]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        1 non-null      object
 1   country     1 non-null      object
 2   latitude    1 non-null      object
 3   longitude   1 non-null      object
 4   Population  1 non-null      object
dtypes: object(5)
memory usage: 168.0+ bytes


In [ ]:
cities_df

,city,country,latitude,longitude,Population
0,Frankfurt,Germany,50.0638,08.4056,773068


In [ ]:
cities_df = cities_df.astype({'latitude':'float','longitude':'float','Population':'int'})

In [ ]:
cities_df['city_id'] = cities_df.index
cities_df

,city,country,latitude,longitude,Population,city_id
0,Frankfurt,Germany,50.0638,8.4056,773068,0


In [ ]:
# The response from the website is stored inside the variable 'Frankfurt'.
Frankfurt = requests.get(f"https://api.openweathermap.org/data/2.5/weather?lat=50.110924&lon=8.682127&appid=d3d5c239f86f1e7dad51cd1cb58bb938&units=metric")
Frankfurt

<Response [200]>

In [ ]:
# Get the status code from the response.
Frankfurt.status_code

200

In [ ]:
# Get the text from the response.
Frankfurt.text

'{"coord":{"lon":8.6821,"lat":50.1106},"weather":[{"id":741,"main":"Fog","description":"fog","icon":"50d"}],"base":"stations","main":{"temp":12.05,"feels_like":11.22,"temp_min":7.97,"temp_max":16.29,"pressure":1023,"humidity":73},"visibility":800,"wind":{"speed":1.03,"deg":0},"clouds":{"all":75},"dt":1696578583,"sys":{"type":2,"id":2077297,"country":"DE","sunrise":1696570329,"sunset":1696611279},"timezone":7200,"id":2925533,"name":"Frankfurt am Main","cod":200}'

In [ ]:
# view the response as a JSON.
Frankfurt.json()

{'coord': {'lon': 8.6821, 'lat': 50.1106},
 'weather': [{'id': 741, 'main': 'Fog', 'description': 'fog', 'icon': '50d'}],
 'base': 'stations',
 'main': {'temp': 12.05,
  'feels_like': 11.22,
  'temp_min': 7.97,
  'temp_max': 16.29,
  'pressure': 1023,
  'humidity': 73},
 'visibility': 800,
 'wind': {'speed': 1.03, 'deg': 0},
 'clouds': {'all': 75},
 'dt': 1696578583,
 'sys': {'type': 2,
  'id': 2077297,
  'country': 'DE',
  'sunrise': 1696570329,
  'sunset': 1696611279},
 'timezone': 7200,
 'id': 2925533,
 'name': 'Frankfurt am Main',
 'cod': 200}

### 3. Day Weather for 5 Days Frankfurt -API

In [ ]:
Frankfurt = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat=50.110924&lon=8.682127&appid=d3d5c239f86f1e7dad51cd1cb58bb938&units=metric")
Frankfurt

<Response [200]>

In [ ]:
weather_json = Frankfurt.json()

In [ ]:
weather_json

In [ ]:
render_json(weather_json)

In [ ]:
# we'll store the information in this dicitonary:

weather_dict = {'city':[],
                'time':[],
                'temp':[],
                'outlook':[],
                'wind_speed':[],
                'pressure':[]}

In [ ]:
# let's begin the loop
for i in weather_json['list']:
    weather_dict['city'].append(weather_json['city']['name'])
    weather_dict['time'].append(i['dt_txt'])
    weather_dict['temp'].append(i['main']['temp'])
    weather_dict['outlook'].append(i['weather'][0]['main'])
    weather_dict['wind_speed'].append(i['wind']['speed'])
    weather_dict['pressure'].append(i['main']['pressure'])

In [ ]:
weather_dict

{'city': ['Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main',
  'Frankfurt am Main'],
 'time': ['2023-10-06 09:00:00',
  '2023-10-06 12:00:00',
  '2023-10-06

In [ ]:
# Now we convert our dictionary to a DataFrame
weather_df = pd.DataFrame(weather_dict)
weather_df

,city,time,temp,outlook,wind_speed,pressure
0,Frankfurt am Main,2023-10-06 09:00:00,12.71,Clouds,2.70,1020
1,Frankfurt am Main,2023-10-06 12:00:00,14.93,Clouds,2.98,1022
2,Frankfurt am Main,2023-10-06 15:00:00,17.82,Clouds,3.10,1022
3,Frankfurt am Main,2023-10-06 18:00:00,16.02,Clouds,1.89,1024
4,Frankfurt am Main,2023-10-06 21:00:00,14.42,Clouds,1.91,1024
5,Frankfurt am Main,2023-10-07 00:00:00,13.34,Clouds,2.19,1024
6,Frankfurt am Main,2023-10-07 03:00:00,12.21,Clouds,2.24,1024
7,Frankfurt am Main,2023-10-07 06:00:00,11.80,Clouds,2.70,1024
8,Frankfurt am Main,2023-10-07 09:00:00,14.75,Clouds,3.42,1024
9,Frankfurt am Main,2023-10-07 12:00:00,20.83,Clouds,4.91,1023


In [ ]:
weather_df['city_id'] = 0

In [ ]:
weather_df

,city,time,temp,outlook,wind_speed,pressure,city_id
0,Frankfurt am Main,2023-10-06 09:00:00,12.71,Clouds,2.70,1020,0
1,Frankfurt am Main,2023-10-06 12:00:00,14.93,Clouds,2.98,1022,0
2,Frankfurt am Main,2023-10-06 15:00:00,17.82,Clouds,3.10,1022,0
3,Frankfurt am Main,2023-10-06 18:00:00,16.02,Clouds,1.89,1024,0
4,Frankfurt am Main,2023-10-06 21:00:00,14.42,Clouds,1.91,1024,0
5,Frankfurt am Main,2023-10-07 00:00:00,13.34,Clouds,2.19,1024,0
6,Frankfurt am Main,2023-10-07 03:00:00,12.21,Clouds,2.24,1024,0
7,Frankfurt am Main,2023-10-07 06:00:00,11.80,Clouds,2.70,1024,0
8,Frankfurt am Main,2023-10-07 09:00:00,14.75,Clouds,3.42,1024,0
9,Frankfurt am Main,2023-10-07 12:00:00,20.83,Clouds,4.91,1023,0


### 4. Airport Data for Frankfurt - API

Frankurt: latitude 50.11 longitude 8.68.

In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/50.11/8.68/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":50.11,"lon":8.68},"count":1,"items":[{"icao":"EDDF","iata":"FRA","name":"Frankfurt-am-Main ","shortName":"Frankfurt-am-Main","municipalityName":"Frankfurt-am-Main","location":{"lat":50.0264,"lon":8.543129},"countryCode":"DE"}]}


In [ ]:
response.json()

{'searchBy': {'lat': 50.11, 'lon': 8.68},
 'count': 1,
 'items': [{'icao': 'EDDF',
   'iata': 'FRA',
   'name': 'Frankfurt-am-Main ',
   'shortName': 'Frankfurt-am-Main',
   'municipalityName': 'Frankfurt-am-Main',
   'location': {'lat': 50.0264, 'lon': 8.543129},
   'countryCode': 'DE'}]}

We can now turn this into a dataframe using .json_normalize()

In [ ]:
Airport_df = pd.DataFrame(pd.json_normalize(response.json()['items']))

In [ ]:
Airport_df

### 5. Arrivals and Depatures at Frankfurt Airport - API

In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
from pytz import timezone

In [ ]:
icao = "EDDF"

In [ ]:
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
tomorrow = (today + timedelta(days=1))

In [ ]:
import requests

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T12:00/{tomorrow}T23:59"

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"true"}

headers = {
	"X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'arrivals': [{'movement': {'airport': {'icao': 'LTFE', 'iata': 'BJV', 'name': 'Bodrum'}, 'scheduledTime': {'utc': '2023-10-07 10:20Z', 'local': '2023-10-07 12:20+02:00'}, 'revisedTime': {'utc': '2023-10-07 10:20Z', 'local': '2023-10-07 12:20+02:00'}, 'terminal': '1', 'quality': ['Basic', 'Live']}, 'number': '4Y 1603', 'status': 'Expected', 'codeshareStatus': 'IsOperator', 'isCargo': False, 'aircraft': {'reg': 'D-AIUX', 'modeS': '3C66B8', 'model': 'Airbus A320'}, 'airline': {'name': 'Discover Airlines'}}, {'movement': {'airport': {'icao': 'KLAS', 'iata': 'LAS', 'name': 'Las Vegas'}, 'scheduledTime': {'utc': '2023-10-07 10:45Z', 'local': '2023-10-07 12:45+02:00'}, 'revisedTime': {'utc': '2023-10-07 10:45Z', 'local': '2023-10-07 12:45+02:00'}, 'terminal': '1', 'quality': ['Basic', 'Live']}, 'number': '4Y 55', 'status': 'Expected', 'codeshareStatus': 'IsOperator', 'isCargo': False, 'aircraft': {'reg': 'D-AIKB', 'modeS': '3C6562', 'model': 'Airbus A330-300'}, 'airline': {'name': 'Discover 

In [ ]:
flights = response.json()

In [ ]:
flights.keys()

dict_keys(['arrivals'])

In [ ]:
len(flights["arrivals"])

348

In [ ]:
flights["arrivals"][0]["movement"]["airport"]["icao"]

'LTFE'

In [ ]:
flights["arrivals"][0]["movement"]["scheduledTime"]['local']

'2023-10-07 12:20+02:00'

In [ ]:
flights["arrivals"][0]["movement"]["terminal"]

'1'

In [ ]:
flights["arrivals"][0]

{'movement': {'airport': {'icao': 'LTFE', 'iata': 'BJV', 'name': 'Bodrum'},
  'scheduledTime': {'utc': '2023-10-07 10:20Z',
   'local': '2023-10-07 12:20+02:00'},
  'revisedTime': {'utc': '2023-10-07 10:20Z',
   'local': '2023-10-07 12:20+02:00'},
  'terminal': '1',
  'quality': ['Basic', 'Live']},
 'number': '4Y 1603',
 'status': 'Expected',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'reg': 'D-AIUX', 'modeS': '3C66B8', 'model': 'Airbus A320'},
 'airline': {'name': 'Discover Airlines'}}

In [ ]:
icao

'EDDF'

In [ ]:
flights_dict = {'ICAO':[],
                'departure_airport':[],
                'local_time':[]}

In [ ]:

        for flight in flights['arrivals']:
            flights_dict['ICAO'].append(icao)
            try:
                flights_dict['departure_airport'].append(flight['movement']['airport']['icao'])
            except:
                flights_dict['departure_airport'].append('unknown')
            try:
                flights_dict['local_time'].append(flight['movement']['scheduledTime']["local"])
            except:
                flights_dict['local_time'].append(pd.NaT)

In [ ]:
flights_df = pd.DataFrame(flights_dict)

In [ ]:
flights_df

### 6. SQL

Pushing City data data base

In [ ]:
# City Data
cities_df

,city,country,latitude,longitude,Population,city_id
0,Frankfurt,Germany,50.0638,8.4056,773068,0


In [ ]:
# City Weather Data
weather_df

In [ ]:
# Airport Data
Airport_df

In [ ]:
# Arrival Depature Data
flights_df

In [ ]:
import sqlite3

In [ ]:
# Create Database
conn = sqlite3.connect('gans_local')

In [ ]:
# To create table
conn.execute('''

CREATE TABLE cities (
    city VARCHAR(100),
    country VARCHAR(100),
    latitude FLOAT,
    longitude FLOAT,
    Population INT,
    city_id INT,
    PRIMARY KEY (city_id)
); '''
)

conn.commit()

In [ ]:
# to see the table
rs = conn.execute('SELECT * FROM cities')
for row in rs:
  print(row)

In [ ]:
cities_df.to_sql('cities', conn, if_exists='replace')

In [ ]:
# Another way
qry = """

SELECT * FROM cities

"""
cities = pd.read_sql_query(qry, conn)
cities

,index,city,country,latitude,longitude,Population,city_id
0,0,Frankfurt,Germany,50.0638,8.4056,773068,0


Pushing City Weather Data

In [ ]:
weather_df.head()

,city,time,temp,outlook,wind_speed,pressure,city_id
0,Frankfurt am Main,2023-10-06 09:00:00,12.71,Clouds,2.70,1020,0
1,Frankfurt am Main,2023-10-06 12:00:00,14.93,Clouds,2.98,1022,0
2,Frankfurt am Main,2023-10-06 15:00:00,17.82,Clouds,3.10,1022,0
3,Frankfurt am Main,2023-10-06 18:00:00,16.02,Clouds,1.89,1024,0
4,Frankfurt am Main,2023-10-06 21:00:00,14.42,Clouds,1.91,1024,0


In [ ]:
# To create table
conn.execute('''

CREATE TABLE weather (
	  weather_id INT AUTO_INCREMENT,
    city VARCHAR(100),
    `time` DATETIME,
    temp FLOAT,
    outlook VARCHAR(100),
    wind_speed FLOAT,
    pressure INT,
    city_id INT NOT NULL,
    PRIMARY KEY (weather_id),
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
); '''
)

conn.commit()

In [ ]:
weather_df.to_sql('weather', conn, if_exists='replace')

40

In [ ]:
# Another way
qry = """

SELECT * FROM weather

"""
cities = pd.read_sql_query(qry, conn)
cities

,index,city,time,temp,outlook,wind_speed,pressure,city_id
0,0,Frankfurt am Main,2023-10-06 09:00:00,12.71,Clouds,2.70,1020,0
1,1,Frankfurt am Main,2023-10-06 12:00:00,14.93,Clouds,2.98,1022,0
2,2,Frankfurt am Main,2023-10-06 15:00:00,17.82,Clouds,3.10,1022,0
3,3,Frankfurt am Main,2023-10-06 18:00:00,16.02,Clouds,1.89,1024,0
4,4,Frankfurt am Main,2023-10-06 21:00:00,14.42,Clouds,1.91,1024,0
5,5,Frankfurt am Main,2023-10-07 00:00:00,13.34,Clouds,2.19,1024,0
6,6,Frankfurt am Main,2023-10-07 03:00:00,12.21,Clouds,2.24,1024,0
7,7,Frankfurt am Main,2023-10-07 06:00:00,11.80,Clouds,2.70,1024,0
8,8,Frankfurt am Main,2023-10-07 09:00:00,14.75,Clouds,3.42,1024,0
9,9,Frankfurt am Main,2023-10-07 12:00:00,20.83,Clouds,4.91,1023,0


Pushing Airport Data


In [ ]:
Airport_df.head()

,icao,city_id
0,EDDF,0


In [ ]:
Airport_df['city_id'] = Airport_df.index

In [ ]:
Airport_df = Airport_df[["icao","city_id"]]

In [ ]:
# To create table
conn.execute('''

CREATE TABLE airports(
	  city_id INT NOT NULL,
    icao VARCHAR(10),
    PRIMARY KEY (icao),
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
); '''
)

conn.commit()

In [ ]:
Airport_df.to_sql('airports', conn, if_exists='replace')

1

In [ ]:
# Another way
qry = """

SELECT * FROM airports

"""
airports = pd.read_sql_query(qry, conn)
airports

,index,icao,city_id
0,0,EDDF,0


Pushing flights data

In [ ]:
flights_df.head()

,ICAO,departure_airport,local_time
0,EDDF,LTFE,2023-10-07 12:20+02:00
1,EDDF,KLAS,2023-10-07 12:45+02:00
2,EDDF,KRSW,2023-10-07 12:25+02:00
3,EDDF,LEPA,2023-10-07 12:30+02:00
4,EDDF,LGKR,2023-10-07 12:40+02:00


In [ ]:
# To create table
conn.execute('''

CREATE TABLE flights(
	  flight_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ICAO VARCHAR(10),
    departure_airport VARCHAR(10),
    local_time DATETIME,
    FOREIGN KEY (ICAO) REFERENCES airports(icao)
); '''
)

conn.commit()

In [ ]:
flights_df.to_sql('flights', conn, if_exists='replace')

348

In [ ]:
# Another way
qry = """

SELECT * FROM flights

"""
airports = pd.read_sql_query(qry, conn)
airports